In [2]:
from utils import cp_detector as cpd
from utils import data_generation as gen
from utils import statistic_calculation as stat
from utils import threshold_calculation as th

import torch.nn as nn
import torch

import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [72]:
dataset_parameters = {
    "dataset_size": 500,
    "seq_len": 100,
    "d": 20,
    "p": 10,
    "distribution": "normal",
    "k": 1,
    "cp": 50,
    "nu": 3,
    "seed": 124
}

cp_parameters = {
    "alpha": 0.05,
    "scan": False,
    "data_based": False,
    "ln": False
}

In [73]:
data_with_cp, data_without_cp, t_cp_idxs = gen.generate_data(**dataset_parameters)

In [74]:
layer_norm = nn.LayerNorm(dataset_parameters["d"], elementwise_affine=True).float()
# data_with_cp = torch.from_numpy(data_with_cp).float()
# data_without_cp = torch.from_numpy(data_without_cp).float()


# data_with_cp_ln = (
#     layer_norm(data_with_cp)
# )
# data_without_cp_ln = (
#     layer_norm(data_without_cp)
# )

data_with_cp_ln = (
    layer_norm(torch.from_numpy(data_with_cp).float()).detach().numpy()
)
data_without_cp_ln = (
    layer_norm(torch.from_numpy(data_without_cp).float()).detach().numpy()
)

In [75]:
from utils import knn_divergence as kl
import warnings
warnings.filterwarnings("ignore")

In [76]:
print('Without LN, F||G')
print('naive', kl.naive_estimator(data_without_cp[0], data_with_cp[0], k=20))
print('scipy', kl.scipy_estimator(data_without_cp[0], data_with_cp[0], k=20))
print('skl_est', kl.skl_estimator(data_without_cp[0], data_with_cp[0], k=20))
print('skl_eff', kl.skl_efficient(data_without_cp[0], data_with_cp[0], k=20))

loss = nn.KLDivLoss(log_target=False, reduction='mean')
print('torch', loss(torch.from_numpy(data_without_cp[0]), torch.from_numpy(data_with_cp[0])).detach().numpy())

Without LN, F||G
naive 0.8182411343112915
scipy 0.8182411343112914
skl_est 0.8182411343112911
skl_eff 0.8182411343112918
torch -0.455888223827306


In [77]:
print('With LN, F||G')
print('naive', kl.naive_estimator(data_without_cp_ln[0], data_with_cp_ln[0], k=20))
print('scipy', kl.scipy_estimator(data_without_cp_ln[0], data_with_cp_ln[0], k=20))
print('skl_est', kl.skl_estimator(data_without_cp_ln[0], data_with_cp_ln[0], k=20))
print('skl_eff', kl.skl_efficient(data_without_cp_ln[0], data_with_cp_ln[0], k=20))
loss = nn.KLDivLoss(log_target=False, reduction='mean')
print('torch', loss(torch.from_numpy(data_without_cp_ln[0]), torch.from_numpy(data_with_cp_ln[0])).detach().numpy())

With LN, F||G
naive 0.02533467026476404
scipy 0.025334551442741368
skl_est 0.025334471241129856
skl_eff 0.025334551442741226
torch -0.4440932


In [78]:
print('Without LN, G||F')
print('naive', kl.naive_estimator(data_with_cp[0], data_without_cp[0], k=20))
print('scipy', kl.scipy_estimator(data_with_cp[0], data_without_cp[0], k=20))
print('skl_est', kl.skl_estimator(data_with_cp[0], data_without_cp[0], k=20))
print('skl_eff', kl.skl_efficient(data_with_cp[0], data_without_cp[0], k=20))
print('torch', loss(torch.from_numpy(data_with_cp[0]), torch.from_numpy(data_without_cp[0])).detach().numpy())

Without LN, G||F
naive 0.15662433668482983
scipy 0.15662433668482958
skl_est 0.15662433668483003
skl_eff 0.15662433668482992
torch -0.595205674063714


In [79]:
print('With LN, G||F')
print('naive', kl.naive_estimator(data_with_cp_ln[0], data_without_cp_ln[0], k=20))
print('scipy', kl.scipy_estimator(data_with_cp_ln[0], data_without_cp_ln[0], k=20))
print('skl_est', kl.skl_estimator(data_with_cp_ln[0], data_without_cp_ln[0], k=20))
print('skl_eff', kl.skl_efficient(data_with_cp_ln[0], data_without_cp_ln[0], k=20))
print('torch', loss(torch.from_numpy(data_with_cp_ln[0]), torch.from_numpy(data_without_cp_ln[0])).detach().numpy())

With LN, G||F
naive 0.629517061309506
scipy 0.629517129028949
skl_est 0.6295170172084718
skl_eff 0.6295171290289491
torch -0.44641107


In [80]:
def kl_divergence(mu1, mu2, sigma_1, sigma_2):
	sigma_diag_1 = np.eye(sigma_1.shape[0]) * sigma_1
	sigma_diag_2 = np.eye(sigma_2.shape[0]) * sigma_2

	sigma_diag_2_inv = np.linalg.inv(sigma_diag_2)

	kl = 0.5 * (np.log(np.linalg.det(sigma_diag_2) / np.linalg.det(sigma_diag_2))
				- mu1.shape[0] + np.trace(np.matmul(sigma_diag_2_inv, sigma_diag_1))
				+ np.matmul(np.matmul(np.transpose(mu2 - mu1), sigma_diag_2_inv), (mu2 - mu1))
				)

	return kl

In [81]:
d = dataset_parameters["d"]
p = dataset_parameters["p"]
k = dataset_parameters["k"]
mu_1 = np.zeros(d)
sigma_1 = np.identity(d)

mu_2 = np.zeros(d)
mu_2[-p:] = (d - 1) * k/d 

sigma_2 = ((d - 1) / d)**2 *np.identity(d)


In [82]:
kl_divergence(mu_1, mu_2, sigma_2, sigma_2)

4.999999999999999

In [83]:
d = dataset_parameters["d"]
p = dataset_parameters["p"]
k = dataset_parameters["k"]
mu_1 = np.zeros(d)
sigma_1 = np.identity(d)

mu_2 = np.zeros(d)
mu_2[-p:] = k

In [84]:
kl_divergence(mu_1, mu_2, sigma_1, sigma_1)

5.0

In [207]:
x = data_without_cp_ln[0]

In [212]:
def estimate_mu_sigma(y):
    mu_hat = np.expand_dims(y.sum(0) / y.shape[0], -1)

    res = 0
    for i in range(y.shape[0]):
        res += (y[i] - mu_hat) * (y[i] - mu_hat).T
    sigma_hat = res / (y.shape[0] - 1)
    return mu_hat.flatten(), sigma_hat

In [213]:
mu_hat, sigma_hat = estimate_mu_sigma(data_without_cp[0])

In [214]:
mu_hat_ln, sigma_hat_ln = estimate_mu_sigma(data_without_cp_ln[0])

In [226]:
mu_hat_ln / mu_hat

array([ 0.72739767, -1.27326259,  1.12118739,  1.62518141,  0.64048046,
        0.77030363,  0.68437955,  2.99446352,  1.17957072,  1.08522984,
        0.84331885,  0.79540898, -0.21919541,  1.26714849,  1.20762244,
        0.82463847,  0.38027637, -2.41119062,  0.74827744,  0.018901  ])

In [229]:
mu_hat_ln

array([ 0.07243666, -0.02243358, -0.13873385,  0.01218617,  0.04842686,
        0.1002454 ,  0.03954205, -0.05337384, -0.16758531, -0.11751797,
        0.14834498,  0.11140257, -0.00927321, -0.08916657, -0.16272879,
        0.12056039,  0.02605991, -0.00798142,  0.08884298,  0.00074645],
      dtype=float32)

In [224]:
sigma_hat / sigma_hat_ln

array([[ 1.13261191e+00, -1.63356314e+00,  1.04270261e+01,
         1.03481785e+00,  1.67292800e+00,  4.77766277e-01,
         5.34046229e-01,  1.98110635e+00, -1.03062774e+00,
         6.32195775e-01,  1.37866141e-01, -1.71221549e+00,
        -1.89496140e+00, -1.09433900e+00,  6.69315681e-01,
        -7.32517501e-01,  2.09136447e-01, -1.54410680e+00,
         8.70227706e-01,  1.77610431e+00],
       [-1.63356314e+00,  1.00646932e+00,  5.37340858e-01,
        -7.85845631e-01, -2.67974093e-01,  7.56369168e-01,
         1.45493158e+00, -1.00101296e+00,  1.25978611e+00,
         5.90541715e-01,  4.14794502e+00,  6.57568554e-01,
         6.38187675e-01, -3.87268966e-01,  2.88979629e+00,
        -1.37669364e+00, -1.17832671e-01,  8.83664556e-01,
         8.96679828e-01, -6.34722241e+01],
       [ 1.04270261e+01,  5.37340858e-01,  1.03958934e+00,
         6.98641346e-01,  6.46993630e-01, -6.23259746e+00,
        -6.92964795e-03,  1.70618684e+00,  2.89065533e+00,
        -2.91550134e-01, -1.4

array([[ 8.42635930e-01, -3.83819714e-02,  1.13512194e-02,
        -6.47054091e-02,  1.18674137e-01, -1.57843053e-01,
        -1.73598766e-01,  7.09673092e-02, -3.01359780e-02,
        -1.92153126e-01, -9.35264751e-02, -4.61357087e-02,
        -3.39687578e-02, -4.45456542e-02, -2.51616210e-01,
        -6.63037226e-02, -8.05425569e-02, -2.38332264e-02,
        -1.55295640e-01,  1.25225767e-01],
       [-3.83819714e-02,  1.06100392e+00, -1.93526611e-01,
         1.88240483e-02, -7.68796727e-02, -1.36816725e-01,
         1.38232157e-01, -4.49110754e-02,  1.46539405e-01,
        -1.31394029e-01,  2.18326822e-02, -2.78757840e-01,
        -1.65485173e-01, -5.86201027e-02,  3.30767743e-02,
        -5.67461997e-02, -4.96105924e-02, -2.02212900e-01,
        -1.73181295e-01, -8.82317312e-04],
       [ 1.13512194e-02, -1.93526611e-01,  1.01275742e+00,
        -9.25110132e-02, -2.60953635e-01, -6.23858208e-03,
        -1.25079498e-01,  5.28685041e-02,  3.23847122e-02,
        -9.78772268e-02, -9.2

In [132]:
sigma_hat

array([[20.455444  , -7.294408  , -2.6192117 , ...,  1.9529359 ,
        -0.81376934, -5.023314  ],
       [-7.294408  , 20.135721  ,  2.1402133 , ...,  3.4562528 ,
        -3.8462355 , -3.675526  ],
       [-2.6192117 ,  2.1402133 , 18.837494  , ...,  3.5155034 ,
        -0.44618866,  3.4433289 ],
       ...,
       [ 1.952936  ,  3.4562526 ,  3.5155034 , ..., 21.797873  ,
         1.7560735 ,  0.7569927 ],
       [-0.8137692 , -3.8462358 , -0.4461894 , ...,  1.7560735 ,
        19.839293  , -9.477086  ],
       [-5.023314  , -3.6755254 ,  3.4433289 , ...,  0.7569927 ,
        -9.477086  , 19.556696  ]], dtype=float32)

In [130]:
sigma_hat_2

array([[ 21.43624129,  -6.83565625,  -2.93054608, ...,   1.08152691,
         -1.33574581,  -4.06779949],
       [ -6.83565625,  21.35449406,   0.43611139, ...,   1.61473433,
         -4.8598766 ,  -1.86610931],
       [ -2.93054608,   0.43611139,  15.99187609, ...,   3.7161146 ,
          1.15126852,   0.92252522],
       ...,
       [  1.08152691,   1.61473433,   3.7161146 , ...,  15.30848921,
          2.47494974,  -0.99240693],
       [ -1.33574581,  -4.8598766 ,   1.15126852, ...,   2.47494974,
         17.88805555, -10.21793166],
       [ -4.06779949,  -1.86610931,   0.92252522, ...,  -0.99240693,
        -10.21793166,  20.80260322]])

0.9025

In [126]:
-7.6783237 * 0.9025

-6.9296871392499995